In [1]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import plotly
import plotly.express as px
from jinja2 import Template

## Parse Data

### Fanduel

In [2]:
data = pd.read_excel('raw_data/fanduel-transaction-data.xlsx', sheet_name=None, header=None)
df = pd.concat(data.values())
df.columns = ['PRODUCT','TRANSACTION ID','DATE','TYPE','DETAILS','SOURCE','CHANGE','BEFORE','BALANCE']
df = df.drop_duplicates()
df = df.reset_index(drop=True)

df['DATE'] = [x.replace('ET', '') for x in df['DATE']]
df['DATE'] = pd.to_datetime(df['DATE'])

df = df[df['TYPE'].isin(['Wager', 'Winnings', 'Bonus', 'Manual Adjustment', 'Deposit', 'Withdraw'])]

df = pd.DataFrame(df)
df['BONUS'] = list(df['TYPE'])
df['BOOK'] = 'Fanduel'

df = df.sort_values('TRANSACTION ID', ascending=False)

df = df[['BOOK', 'DATE', 'TYPE', 'BONUS', 'CHANGE', 'BALANCE']]
df.columns = ['Book', 'Time', 'Action', 'Bonus', 'Change', 'Balance']

df['Bonus'] = ['Bonus' if x in ['Bonus', 'Manual Adjustment'] else '' for x in df['Bonus']]

action_map = {'Wager':'Wager', 'Winnings':'Winnings', 'Bonus':'Winnings', 'Manual Adjustment':'Winnings', 
              'Deposit':'Deposit', 'Withdraw':'Withdraw'}
df['Action'] = [action_map[x] for x in df['Action']]

df = df.sort_values('Time')
df = df.reset_index(drop=True)

In [3]:
# fix balance errors
for i in range(1, df.shape[0]):
    r_prev = df.iloc[i-1]
    r_curr = df.iloc[i]

    if (abs(r_curr['Change']) <= 150) and (r_prev['Balance'] - r_curr['Balance'] > 150):
        df.iloc[i,5] = r_prev['Balance'] + r_curr['Change']

fanduel = df
fanduel.to_csv('processed_data/fanduel.csv', index=False)

### Draftkings

In [4]:
data = pd.read_excel('raw_data/draftkings-transaction-data.xlsx', sheet_name=None, header=None)
cells = pd.concat(data.values())

data = {
    'Date': [],
    'Time': [],
    'Category': [],
    'Description': [],
    'ID Ref': [],
    'ID': [],
    'Product': [],
    'Change': [],
    'Balance': []
}


for i in range(len(cells[0])):
    c = cells[0].iloc[i]
    
    if i % 9 == 0:
        data['Date'].append(c)
    elif i % 9 == 1:
        data['Time'].append(c)
    elif i % 9 == 2:
        data['Category'].append(c)
    elif i % 9 == 3:
        data['Description'].append(c)
    elif i % 9 == 4:
        data['ID Ref'].append(c)
    elif i % 9 == 5:
        data['ID'].append(c)
    elif i % 9 == 6:
        data['Product'].append(c)
    elif i % 9 == 7:
        data['Change'].append(c)
    elif i % 9 == 8:
        data['Balance'].append(c)


df = pd.DataFrame(data)
df = df.drop_duplicates()

df['Book'] = 'Draftkings'

df['Datetime'] = [str(df['Date'].iloc[i].date()) + ' ' + df['Time'].iloc[i] for i in range(df.shape[0])]
df['Datetime'] = pd.to_datetime(df['Datetime'])

df['Bonus'] = ['Bonus' if x in ['Promotions and Rewards'] else '' for x in df['Category']]

action_map = {'Wagers and Entry Fees and Purchases':'Wager', 'Winnings':'Winnings', 'Promotions and Rewards':'Winnings',
              'Deposits':'Deposit', 'Withdraws':'Withdraw'}
df['Action'] = [action_map[x] for x in df['Category']]

df = df[['Book', 'Datetime', 'Action', 'Bonus', 'Change', 'Balance']]
df.columns = ['Book', 'Time', 'Action', 'Bonus', 'Change', 'Balance']

df = df.sort_values('Time')
df = df.reset_index(drop=True)

draftkings = df
draftkings.to_csv('processed_data/draftkings.csv', index=False)

### Caesars

In [5]:
data = pd.read_excel('raw_data/caesars-transaction-data.xlsx', sheet_name=None, header=None)
cells = pd.concat(data.values())[0]

data = {
    'Time': [],
    'Type': [],
    'Amount': [],
    'Balance': [],
    'Status': []
}

i = 0
curr_date = None

while True:

    if not i < len(cells):
        break

    c = cells.iloc[i]

    if '20' in str(c) and str(c).count('-') >= 2 and str(c).count(':') >= 2:
        curr_date = pd.to_datetime(c).date()
        i += 1
        continue

    if c == 'Time':
        data['Time'].append(str(curr_date) + ' ' + cells.iloc[i+1])
    elif c == 'Type':
        data['Type'].append(cells.iloc[i+1])
    elif c == 'Amount':
        data['Amount'].append(cells.iloc[i+1])
    elif c == 'Balance':
        data['Balance'].append(cells.iloc[i+1])
    elif c == 'Status':
        data['Status'].append(cells.iloc[i+1])
        assert len(data['Time']) == len(data['Type']) == \
               len(data['Amount']) == len(data['Balance'])

    i += 1


In [6]:
df = pd.DataFrame(data)

df = df.drop_duplicates()
df = df[df['Status'] == 'Approved']

df['Book'] = 'Caesars'
df['Time'] = [x.replace('EDT', '') for x in df['Time']]
df['Time'] = pd.to_datetime(df['Time'])

df['Bonus'] = ['Bonus' if x in ['Bonus'] else '' for x in df['Type']]

action_map = {'Bet':'Wager', 'Payout':'Winnings', 'Bonus':'Winnings', 'Void': 'Winnings',
              'Deposit':'Deposit', 'Withdraws':'Withdraw'}
df['Action'] = [action_map[x] for x in df['Type']]

# fix bonus balance sync
new_balance = list(df['Balance'])
for i in range(1, df.shape[0]):
    p = df.iloc[i-1]
    r = df.iloc[i]
    if p['Time'] == r['Time'] and abs(p['Amount']) == abs(r['Amount']):
        if p['Bonus'] == 'Bonus':
            new_balance[i] = r['Balance']
            new_balance[i-1] = r['Balance']
        elif r['Bonus'] == 'Bonus':
            new_balance[i] = p['Balance']
            new_balance[i-1] = p['Balance']

df['New Balance'] = new_balance
    
df = df[['Book', 'Time', 'Action', 'Bonus', 'Amount', 'New Balance']]
df.columns = ['Book', 'Time', 'Action', 'Bonus', 'Change', 'Balance']

df = df.sort_values('Time')
df = df.reset_index(drop=True)

caesars = df
caesars.to_csv('processed_data/caesars.csv', index=False)

### Net Total

In [7]:
balance = {
    'Fanduel': 0,
    'Draftkings': 0,
    'Caesars': 0
}

df = pd.concat([fanduel, draftkings, caesars])
df = df.sort_values('Time')
df = df.reset_index(drop=True)

net_df = df.copy()

net_balance = []
for r in df.iloc:
    balance[r['Book']] = r['Balance']
    net_balance.append(sum(balance.values()))

net_df['Net Balance'] = net_balance

net_df.columns = ['Book', 'Time', 'Action', 'Bonus', 'Change', 'Book Balance', 'Net Balance']
net_df['Change'] = net_df['Change'].round(2)
net_df['Book Balance'] = net_df['Book Balance'].round(2)
net_df['Net Balance'] = net_df['Net Balance'].round(2)
net_df.to_csv('processed_data/net.csv', index=False)
net_df

,Book,Time,Action,Bonus,Change,Book Balance,Net Balance
0,Fanduel,2021-04-01 14:48:00,Deposit,,50.00,50.00,50.00
1,Fanduel,2021-04-01 15:11:00,Wager,,-5.00,45.00,45.00
2,Fanduel,2021-04-01 15:23:00,Winnings,Bonus,50.00,95.00,95.00
3,Fanduel,2021-04-01 15:29:00,Wager,,-10.00,85.00,85.00
4,Fanduel,2021-04-02 12:45:00,Winnings,Bonus,10.00,95.00,95.00
...,...,...,...,...,...,...,...
1482,Fanduel,2022-05-12 11:36:00,Wager,,-11.00,1025.00,2906.70
1483,Caesars,2022-05-12 16:32:00,Wager,,-5.96,974.00,2900.74
1484,Draftkings,2022-05-12 21:48:00,Winnings,,22.00,923.74,2922.74
1485,Caesars,2022-05-13 00:12:00,Winnings,,3.43,977.43,2926.17


## Compute Statistics

In [8]:
# helper format functions
def dollars(x):
    if x >= 0:
        return '$%.2f' % round(x, 2)
    else:
        return '-$%.2f' % round(abs(x), 2)


def dollar_diff(x):
    if x >= 0:
        return '+$%.2f' % round(x, 2)
    else:
        return '-$%.2f' % round(abs(x), 2)

def round2(x):
    return round(x, 2)


In [9]:
month_names = ['', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'June', 'July', 'Aug', 'Sept', 'Oct', 'Nov', 'Dec']
last_entry = net_df['Time'].max()
last_update = '%s %d, %d' % (month_names[last_entry.month], last_entry.day, last_entry.year)
last_update

'May 13, 2022'

In [10]:
# net stats
deposits = abs(net_df[net_df['Action'] == 'Deposit']['Change'].sum())
withdraws = abs(net_df[net_df['Action'] == 'Withdraw']['Change'].sum())
wagers = abs(net_df[net_df['Action'] == 'Wager']['Change'].sum())
winnings = abs(net_df[net_df['Action'] == 'Winnings']['Change'].sum())
net_profit = winnings + withdraws - wagers - deposits

net_stats_table = pd.DataFrame(
    [
        ['Deposits', dollars(deposits)],
        ['Withdraws', dollars(withdraws)],
        ['Wagers', dollars(wagers)],
        ['Winnings', dollars(winnings)],
        ['Net Profit', dollars(net_profit)]
    ], 
    columns=['Name', 'Value']
)
net_stats_table

,Name,Value
0,Deposits,$300.00
1,Withdraws,$0.00
2,Wagers,$8400.22
3,Winnings,$11041.69
4,Net Profit,$2341.47


In [11]:
# book balances

total_balance = dollars(net_df[net_df['Time'] == net_df['Time'].max()]['Net Balance'].values[0])

book_balance_table = pd.DataFrame(
    [
        ['Fanduel', dollars(fanduel[fanduel['Time'] == fanduel['Time'].max()]['Balance'].values[0])],
        ['Draftkings', dollars(draftkings[draftkings['Time'] == draftkings['Time'].max()]['Balance'].values[0])],
        ['Caesars', dollars(caesars[caesars['Time'] == caesars['Time'].max()]['Balance'].values[0])]
    ], 
    columns=['Name', 'Value']
)
print('Total balance:', total_balance)
book_balance_table

Total balance: $2941.47


,Name,Value
0,Fanduel,$1025.00
1,Draftkings,$939.04
2,Caesars,$977.43


In [12]:
# career bets

total_bets = net_df[net_df['Action'] == 'Wager']['Action'].count()
avg_bet = net_df[net_df['Action'] == 'Wager']['Change'].abs().sum() / total_bets
avg_payout = net_df[net_df['Action'] == 'Winnings']['Change'].abs().sum() / total_bets

career_bets_table = pd.DataFrame(
    [
        ['Total Bets Placed', str(total_bets)],
        ['Average Bet Amount', dollars(avg_bet)]
    ],
    columns=['Name', 'Value']
)

career_bets_table

,Name,Value
0,Total Bets Placed,876
1,Average Bet Amount,$9.63


In [13]:
# 90 day stats
start = pd.to_datetime(last_entry - timedelta(days=90))
sub_df = net_df[net_df['Time'] > start]

deposits = abs(sub_df[sub_df['Action'] == 'Deposit']['Change'].sum())
withdraws = abs(sub_df[sub_df['Action'] == 'Withdraw']['Change'].sum())
wagers = abs(sub_df[sub_df['Action'] == 'Wager']['Change'].sum())
winnings = abs(sub_df[sub_df['Action'] == 'Winnings']['Change'].sum())
profit = winnings + withdraws - wagers - deposits

last_90_days_table = pd.DataFrame(
    [
        ['Total Profit', dollars(profit)],
        ['Per Week Average', dollars(profit / 90 * 7)],
        ['Per Day Average', dollars(profit / 90)]
    ],
    columns=['Name', 'Value']
)

last_90_days_table

,Name,Value
0,Total Profit,$1491.01
1,Per Week Average,$115.97
2,Per Day Average,$16.57


## Generate Plots

In [14]:
# color sequences
market_colors = ["#1493FF", "#61B510", "#C5A359"]
theme_colors = ['#1F77B4', '#2CA02C', 'rgb(242, 183, 1)']
red_col = ['#D62728']
grey_col = ['#7F7F7F']

In [15]:
book_balance_table = pd.DataFrame(
    [
        ['Fanduel', round2(fanduel[fanduel['Time'] == fanduel['Time'].max()]['Balance'].values[0])],
        ['Draftkings', round2(draftkings[draftkings['Time'] == draftkings['Time'].max()]['Balance'].values[0])],
        ['Caesars', round2(caesars[caesars['Time'] == caesars['Time'].max()]['Balance'].values[0])]
    ], 
    columns=['Book', 'Balance']
)

fig = px.bar(
    book_balance_table, 
    x="Balance", y="Book", 
    orientation='h', 
    color='Book',
    text_auto=True,
    template='simple_white',
    color_discrete_sequence=theme_colors,
    height=300
)
fig.update_xaxes(visible=False, showticklabels=False)
fig.update_layout(showlegend=False)
fig.update_traces(textfont_size=13, textangle=0, textposition="outside", cliponaxis=False)
fig.update_layout(xaxis_tickprefix = '$', xaxis_tickformat = ',.2f')
book_balance_bars = plotly.offline.plot(fig, include_plotlyjs=False, output_type='div', config= dict(displayModeBar = False))

In [16]:
# monthly stats
month_names = ['', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'June', 'July', 'Aug', 'Sept', 'Oct', 'Nov', 'Dec']

months = [pd.to_datetime(net_df['Time'].max() - timedelta(days=31*i)) for i in range(6)]

data = []
for d in months:
    sub_df = net_df[[x.year == d.year for x in net_df['Time']]]
    sub_df = sub_df[[x.month == d.month for x in sub_df['Time']]]

    change = sub_df[sub_df['Time'] == sub_df['Time'].max()]['Net Balance'].iloc[0] - \
             sub_df[sub_df['Time'] == sub_df['Time'].min()]['Net Balance'].iloc[0]

    deposits = abs(sub_df[sub_df['Action'] == 'Deposit']['Change'].sum())
    withdraws = abs(sub_df[sub_df['Action'] == 'Withdraw']['Change'].sum())
    profit = change + withdraws - deposits

    t_name = '%s %d' % (month_names[d.month], d.year)
    data.append([t_name, round2(profit)])
    
monthly_profit_table = pd.DataFrame(
    data,
    columns=['Month', 'Net Profit']
)

fig = px.bar(
    monthly_profit_table, 
    x="Net Profit", y="Month", 
    orientation='h', 
    text_auto=True,
    template='simple_white',
    color_discrete_sequence=theme_colors,
    height=400
)
fig.update_xaxes(visible=False, showticklabels=False)
fig.update_layout(showlegend=False)
fig.update_traces(textfont_size=12, textangle=0, textposition="outside", cliponaxis=False)
fig.update_layout(xaxis_tickprefix = '$', xaxis_tickformat = ',.2f')
monthly_profit_bars = plotly.offline.plot(fig, include_plotlyjs=False, output_type='div', config= dict(displayModeBar = False))

In [17]:
# all time
fig = px.line(
    net_df, 
    x="Time", y="Book Balance", 
    color='Book',
    template='simple_white',
    color_discrete_sequence=theme_colors,
)
fig.update_layout(yaxis_tickprefix = '$')
book_balance_all_plot = plotly.offline.plot(fig, include_plotlyjs=False, output_type='div', config= dict(displayModeBar = False))

In [18]:
# last 90 days
sub_df = net_df[net_df['Time'] > net_df['Time'].max() - timedelta(days=90)]
sub_df

fig = px.line(
    sub_df, 
    x="Time", y="Book Balance", 
    color='Book',
    template='simple_white',
    color_discrete_sequence=theme_colors,
)
fig.update_layout(yaxis_tickprefix = '$')
book_balance_plot = plotly.offline.plot(fig, include_plotlyjs=False, output_type='div', config= dict(displayModeBar = False))

In [19]:
# total balance - all time
fig = px.line(
    net_df, 
    x="Time", y="Net Balance", 
    template='simple_white',
    color_discrete_sequence=red_col,
)
fig.update_layout(yaxis_tickprefix = '$')
net_balance_all_plot = plotly.offline.plot(fig, include_plotlyjs=False, output_type='div', config= dict(displayModeBar = False))

In [20]:
# total balance - 90 days
sub_df = net_df[net_df['Time'] > net_df['Time'].max() - timedelta(days=90)]
sub_df

fig = px.line(
    sub_df, 
    x="Time", y="Net Balance", 
    template='simple_white',
    color_discrete_sequence=red_col,
)
fig.update_layout(yaxis_tickprefix = '$')
net_balance_plot = plotly.offline.plot(fig, include_plotlyjs=False, output_type='div', config= dict(displayModeBar = False))

## Render Webpage

In [21]:
# main page (index.html)
with open('templates/main.html', 'r') as file:
    template_text = file.read()

template = Template(template_text)

html = template.render(
    last_update=last_update,
    total_balance=total_balance,
    net_stats_table=net_stats_table,
    book_balance_bars=book_balance_bars,
    career_bets_table=career_bets_table,
    last_90_days_table=last_90_days_table,
    monthly_profit_bars=monthly_profit_bars,
    book_balance_all_plot=book_balance_all_plot,
    book_balance_plot=book_balance_plot,
    net_balance_all_plot=net_balance_all_plot,
    net_balance_plot=net_balance_plot
)

with open("index.html", "w") as html_file:
    html_file.write(html)